# Seção 14 - Regressão Linear (Desafio_Auto Imports Database)
Para esta seção, usarei a base de dados descrita abaixo:

1. Title: 1985 Auto Imports Database

2. Source Information:
   
   -- Creator/Donor: Jeffrey C. Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
   
   -- Date: 19 May 1987
   
   -- Sources:
     
     1) 1985 Model Import Car and Truck Specifications, 1985 Ward's Automotive Yearbook.
     
     2) Personal Auto Manuals, Insurance Services Office, 160 Water Street, New York, NY 10038 
     
     3) Insurance Collision Report, Insurance Institute for Highway Safety, Watergate 600, Washington, DC 20037
        
A base acima citadada pode ser acessada no link: https://archive.ics.uci.edu/ml/datasets/Automobile.

In [84]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from plotting import (multiple_histograms_plot,
                      bar_plot_with_categorical,
                      plot_confusion_matrix,
                      plot_confusion_matrix_2,
                      plot_roc)

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Lidar com validação de modelos.
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [85]:
nomeDaBase = "../bases/carros/imports-85.csv"
df = pd.read_csv(nomeDaBase, sep=",", encoding="utf-8")

In [86]:
df.sample(5).T

,139,196,4,86,153
symboling,2,-2,2,1,0
normalized-losses,83,103,164,125,77
make,subaru,volvo,audi,mitsubishi,toyota
fuel-type,gas,gas,gas,gas,gas
aspiration,std,std,std,std,std
num-of-doors,two,four,four,four,four
body-style,hatchback,sedan,sedan,sedan,wagon
drive-wheels,fwd,rwd,4wd,fwd,fwd
engine-location,front,front,front,front,front
wheel-base,93.7,104.3,99.4,96.3,95.7


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    205 non-null object
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         205 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 205 non-null object
stroke               205 non-null object
compression-ratio    205 non-null float64
horsepower           205 non-nul

In [88]:
df.describe()

,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


### Observação:
Na base os valores com a string "?" indicam *nulls*. Eu creio que isso pode nos levar a conclusões errôneas como estimar errôneamente a quantidade de *nulls*. Vou substituir os "?" por "nulls".

In [89]:
df.replace(to_replace="?", value=np.nan, inplace=True)

**ATENÇÃO:** Perceba que agora alguns dos atributos possuem nulos.

Agora, vou transformar os atributos que ainda estão como objects em floats.

In [90]:
lista_converter = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

In [91]:
for item in lista_converter:
    df[item] = df[item].astype("float64")

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    164 non-null float64
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         203 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression-ratio    205 non-null float64
horsepower           203 non-

In [93]:
# state_dummies = pd.get_dummies(x_train["addr_state"],drop_first=True)
# x_train = pd.concat([x_train,state_dummies],axis=1)

df_teste = df

In [94]:
lista_atributos_categoricos = ["make", "fuel-type", "aspiration", "num-of-doors",
                               "body-style", "drive-wheels", "engine-location", "engine-type",
                               "num-of-cylinders", "fuel-system"]

In [95]:
state_dummies = pd.get_dummies(df_teste[lista_atributos_categoricos])
df_teste.drop(lista_atributos_categoricos, axis=1, inplace=True)

In [96]:
df_teste = pd.concat([df_teste, state_dummies], axis=1)

In [97]:
df_teste.head()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,make_alfa-romero,make_audi,make_bmw,make_chevrolet,make_dodge,make_honda,make_isuzu,make_jaguar,make_mazda,make_mercedes-benz,make_mercury,make_mitsubishi,make_nissan,make_peugot,make_plymouth,make_porsche,make_renault,make_saab,make_subaru,make_toyota,make_volkswagen,make_volvo,fuel-type_diesel,fuel-type_gas,aspiration_std,aspiration_turbo,num-of-doors_four,num-of-doors_two,body-style_convertible,body-style_hardtop,body-style_hatchback,body-style_sedan,body-style_wagon,drive-wheels_4wd,drive-wheels_fwd,drive-wheels_rwd,engine-location_front,engine-location_rear,engine-type_dohc,engine-type_dohcv,engine-type_l,engine-type_ohc,engine-type_ohcf,engine-type_ohcv,engine-type_rotor,num-of-cylinders_eight,num-of-cylinders_five,num-of-cylinders_four,num-of-cylinders_six,num-of-cylinders_three,num-of-cylinders_twelve,num-of-cylinders_two,fuel-system_1bbl,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,3,NaN,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21.0,27.0,13495.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,3,NaN,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21.0,27.0,16500.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,1,NaN,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154.0,5000.0,19.0,26.0,16500.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102.0,5500.0,24.0,30.0,13950.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18.0,22.0,17450.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


## Ponto de Atenção
Nesse ponto vou tenho duas alternativas:

1. Remover os registros que possuem nulos; ou
2. Substituir os valores nulos por outros valores coerentes com a distribuição observada naquele atributo.

Vou comparar os resultados das duas abordagens.

### 1) Removendo os registros que possuem nulos

In [98]:
df_teste_semNulos = df_teste.dropna(axis=0, how="any")

In [99]:
queda = 100*(len(df_teste)-len(df_teste_semNulos))/len(df_teste)
print("Queda de {:.4}% de registros, passamos de {} para {} registros.".
      format(queda, len(df_teste), len(df_teste_semNulos)))

Queda de 21.95% de registros, passamos de 205 para 160 registros.


**A queda no número de registros foi elevada.** Isso indica que a abordagem de remoção de registros pode não ser boa. Vamos continuar com essa abordagem e comparar com a segunda abordagem.

In [100]:
x = df_teste_semNulos.drop("price", axis=1)
x = x.iloc[:,:].values
y = df_teste_semNulos.price.values

print(type(x), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [101]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [116]:
n = x_test.shape[0]
k = x_test.shape[1]

print(n, k)

32 75


In [133]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("RMSE: {:.5}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("MAE: {:.5}".format(mean_absolute_error(y_test, y_pred)))
# print("MAPE:, ", mean_absolute_percentage_error(y_test, y_pred))
print("R2: {:.5}".format(r2_score(y_test, y_pred)))

################################################################################
# Cálculo do R2 ajustado:
def r2_ajustado(x_train, y_test, y_pred):
    """
    r2_ajustado retorna o R² Ajustado e recebe como argumento as séries com o
    valor alvo teste e o predito.
    """
    n = x_train.shape[0]
    k = x_train.shape[1]
    return (1-((n-1)/(n-(k+1)))*(1-r2_score(y_test, y_pred)))
################################################################################

print("R2 Ajustado: {:.5}".format(r2_ajustado(x_train, y_test, y_pred)))
print("MSE: {:.5}".format(mean_squared_error(y_test, y_pred)))

RMSE: 1671.4
MAE: 1237.9
R2: 0.78521
R2 Ajustado: 0.47543
MSE: 2.7936e+06


**Testando um registro aleatório:**

In [112]:
from random import seed, randint
seed = 42
aleatorio = randint(0,len(x))
resgitro_teste = x[aleatorio]
print("Registro n.º: {}".format(aleatorio))

Registro n.º: 108


In [113]:
y_pred_teste = regressor.predict(resgitro_teste.reshape(1, -1))

print("Valor real: {:.2f}".format(y[aleatorio]))
print("Valor predito: {:.2f}".format(y_pred_teste[0]))

diferença = abs(y[aleatorio]-y_pred_teste[0])
print("Diferença de {:.2f} ou {:.2f}%".format(diferença, 100*diferença/y[aleatorio]))

Valor real: 8013.00
Valor predito: 8896.37
Diferença de 883.37 ou 11.02%
